# Câu 1: Phân loại tập dữ liệu Iris với 4 thuộc tính

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
from scipy.stats import norm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [2]:
iris = sns.load_dataset("iris")
iris = iris.rename(index=str, columns={'sepal_length':'1_sepal_length','sepal_width':'2_sepal_width', 'petal_length':'3_petal_length', 'petal_width':'4_petal_width'})
print('Done')

Done


Lấy dữ liệu thủ công với mỗi loại hoa lấy 80% là tập training, 20% là tập test:

In [3]:
df1 = iris[["1_sepal_length", "2_sepal_width", "3_petal_length", "4_petal_width", "species"]]
X = df1[["1_sepal_length", "2_sepal_width", "3_petal_length", "4_petal_width"]]
y = df1['species']
df_test = pd.concat([df1[40:50], df1[90:100], df1[140:150]])
X_train = pd.concat([df1[0:40], df1[50:90], df1[100:140]]).drop(['species'], axis = 1)
X_test = pd.concat([df1[40:50], df1[90:100], df1[140:150]]).drop(['species'], axis = 1)


y_labels = df1.iloc[:, 4].replace({'setosa':0, 'versicolor':1, 'virginica':2}).copy()
y_train = pd.concat([y_labels[0:40], y_labels[50:90], y_labels[100:140]])
y_test = pd.concat([y_labels[40:50], y_labels[90:100], y_labels[140:150]])
print('Done')


Done


/tmp/ipykernel_15758/2048182279.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_labels = df1.iloc[:, 4].replace({'setosa':0, 'versicolor':1, 'virginica':2}).copy()


Hàm thực hiện thuật toán NB:

Ta đã xác định được rằng các trường trong tập dữ liệu là đại lượng ngẫu nhiên liên tục, với giả thiết các trường tuân theo phân phối chuẩn và các trường độc lập với nhau, ta có công thức tính score như ở dưới

In [4]:
def predict_NB_gaussian_class(X, mu_list, std_list, pi_list):
  scores_list = []
  classes = len(mu_list)
  for c in range(classes):
    score = norm.pdf(x = X[0], loc = mu_list[c][0][0], scale = std_list[c][0][0]) * norm.pdf(x = X[1], loc = mu_list[c][0][1], scale = std_list[c][0][1]) * norm.pdf(x = X[2], loc = mu_list[c][0][2], scale = std_list[c][0][2]) * norm.pdf(x = X[3], loc = mu_list[c][0][3], scale = std_list[c][0][3]) * pi_list[c]
    scores_list.append(score)
  
  return np.argmax(scores_list)
print('Done')

Done


Thực hiện việc lấy giá trị trung bình, độ lệch chuẩn, xác suất của từng lớp pi

In [5]:
mu_list = np.split(df1.groupby('species').mean().values,[1,2])
std_list = np.split(df1.groupby('species').std().values,[1,2], axis = 0)
pi_list = df1.iloc[:,4].value_counts().values / len(df1)
print(mu_list)
print('Done')

[array([[5.006, 3.428, 1.462, 0.246]]), array([[5.936, 2.77 , 4.26 , 1.326]]), array([[6.588, 2.974, 5.552, 2.026]])]
Done


Thực hiện thuật toán NB trên tập test

In [6]:
y_pred = np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  y_pred[i] = predict_NB_gaussian_class(X_test.iloc[i, :], mu_list, std_list, pi_list)

  

/tmp/ipykernel_15758/3962004731.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  score = norm.pdf(x = X[0], loc = mu_list[c][0][0], scale = std_list[c][0][0]) * norm.pdf(x = X[1], loc = mu_list[c][0][1], scale = std_list[c][0][1]) * norm.pdf(x = X[2], loc = mu_list[c][0][2], scale = std_list[c][0][2]) * norm.pdf(x = X[3], loc = mu_list[c][0][3], scale = std_list[c][0][3]) * pi_list[c]


Tính accuracy giữa kết quả dự đoán và kết quả thực tế

In [7]:
print('Accuracy: ', accuracy_score(y_pred, y_test))
print('Recall :', recall_score(y_pred, y_test, average='macro'))
print('Precision: ', precision_score(y_pred, y_test, average='macro'))
print('F1 Score: ', f1_score(y_pred, y_test, average='macro'))

Accuracy:  1.0
Recall : 1.0
Precision:  1.0
F1 Score:  1.0


## Sử dụng thư viện sklearn

In [11]:
X_train_lib, X_test_lib, y_train_lib, y_test_lib = train_test_split(X, y, train_size=0.8,)

In [12]:
model = GaussianNB()
model.fit(X_train_lib, y_train_lib)
y_pred = model.predict(X_test_lib)
print('Accuracy: ', accuracy_score(y_pred, y_test_lib))
print('Recall :', recall_score(y_pred, y_test_lib, average='macro'))
print('Precision: ', precision_score(y_pred, y_test_lib, average='macro'))
print('F1 Score: ', f1_score(y_pred, y_test_lib, average='macro'))

Accuracy:  1.0
Recall : 1.0
Precision:  1.0
F1 Score:  1.0


# Câu 2: Phân loại bệnh nhân ung thư

Đọc dữ liệu từ file, xử lý dữ liệu và phân chia dữ liệu

In [13]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week_1/data/breast-cancer-wisconsin.data', header=None)
df = df.drop(0, axis = 1) # Bỏ cột 0 là mã số mẫu vì không liên quan tới bài toán
#Ép các cột (trừ cột label về số)
for col in df.columns[:-1]:
  df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.fillna(df.mean()) # Ghi đè các giá trị Nan bằng trung bình với giả thiết các trường dữ liệu được phân phối theo phân bố chuẩn
benign = df[df[10] == 2]
malignant = df[df[10] == 4]
#Lấy 80 dòng ngẫu nhiên của lành tính và 40 dòng ngẫu nhiên của ác tính
benign_test = benign.sample(n = 80, random_state=17)
malignant_test = malignant.sample(n = 40, random_state=17)

#Lấy phần còn lại của lành tính và ác tính làm tập train
benign_train = benign.drop(benign_test.index)
malignant_train = malignant.drop(malignant_test.index)
# Gộp train và test vào làm X_train, X_test, y_train, y_test
X_train = pd.concat([benign_train.iloc[:, :9], malignant_train.iloc[:, :9]], axis=0)
y_train = pd.concat([benign_train.iloc[:, 9], malignant_train.iloc[:, 9]], axis=0)
X_test = pd.concat([benign_test.iloc[:, :9], malignant_test.iloc[:, :9]], axis=0)
y_test = pd.concat([benign_test.iloc[:, 9], malignant_test.iloc[:, 9]], axis=0)



print(len(X_train), len(X_test), len(y_train), len(y_test))
# print(X_train)
print(df.dtypes)

579 120 579 120
1       int64
2       int64
3       int64
4       int64
5       int64
6     float64
7       int64
8       int64
9       int64
10      int64
dtype: object


Hàm dự đoán Naive Bayes

In [14]:
def predict_NB_gaussian_class(X,mu_list,std_list,pi_list):
#Returns the class for which the Gaussian Naive Bayes objective function has greatest value
    scores_list = []
    classes = len(mu_list)
    for p in range(classes):
        score = (norm.pdf(x = X[0], loc = mu_list[p][0][0], scale = std_list[p][0][0])
        * norm.pdf(x = X[1], loc = mu_list[p][0][1], scale = std_list[p][0][1])
        * norm.pdf(x = X[2], loc = mu_list[p][0][2], scale = std_list[p][0][2])
        * norm.pdf(x = X[3], loc = mu_list[p][0][3], scale = std_list[p][0][3] )
        * norm.pdf(x = X[4], loc = mu_list[p][0][4], scale = std_list[p][0][4])
        * norm.pdf(x = X[5], loc = mu_list[p][0][5], scale = std_list[p][0][5])
        * norm.pdf(x = X[6], loc = mu_list[p][0][6], scale = std_list[p][0][6])
        * norm.pdf(x = X[7], loc = mu_list[p][0][7], scale = std_list[p][0][7])
        * norm.pdf(x = X[8], loc = mu_list[p][0][8], scale = std_list[p][0][8] )
        * pi_list[p])
        scores_list.append(score)
        
    return 2 if np.argmax(scores_list) == 0 else 4

Tính giá trị trung bình, độ lệch chuẩn, xác suất theo từng lớp của dữ liệu

In [16]:
mu_list = np.split(df.groupby(9).mean().values, [1])
std_list = np.split(df.groupby(9).std().values, [1], axis=0)
pi_list = df.iloc[:, 9].value_counts().values / len(df)
print(mu_list)
print(std_list)
print(pi_list)

[array([[3.85146805, 2.44905009, 2.57858377, 2.2193437 , 2.75474957,
        2.78879878, 3.00345423, 2.18480138, 2.46286701]]), array([[ 7.08571429,  4.88571429,  4.8       ,  4.14285714,  3.8       ,
         6.91428571,  4.77142857,  5.54285714,  3.54285714],
       [ 7.21212121,  6.6969697 ,  6.48484848,  5.60606061,  5.81818182,
         7.33333333,  5.63636364,  5.81818182,  3.87878788],
       [ 7.16666667,  6.66666667,  5.66666667,  7.5       ,  5.5       ,
         7.91666667,  5.33333333,  7.33333333,  4.        ],
       [ 6.33333333,  6.83333333,  7.        ,  5.66666667,  5.        ,
         7.33333333,  5.83333333,  4.        ,  3.66666667],
       [ 8.        ,  8.33333333,  7.66666667,  5.        ,  5.33333333,
        10.        ,  7.33333333,  8.        ,  4.        ],
       [ 6.        ,  6.88888889,  7.11111111,  7.33333333,  6.66666667,
         7.55555556,  6.22222222,  6.88888889,  3.77777778],
       [ 6.75      ,  6.5       ,  7.125     ,  5.375     ,  5.25   

Thực hiện tính toán phân loại trên tập train và tập test

Tập train:

In [17]:
y_train_pred = np.zeros(X_train.shape[0])
for i in range(X_train.shape[0]):
    x = X_train.iloc[i].values
    y_train_pred[i] = predict_NB_gaussian_class(x, mu_list, std_list, pi_list)
print('Accuracy (train): ', accuracy_score(y_train, y_train_pred))
print('Precision (train): ', precision_score(y_train, y_train_pred, pos_label=4))
print('Recall (train): ', recall_score(y_train, y_train_pred, pos_label=4))

Accuracy (train):  0.9309153713298791
Precision (train):  0.9707602339181286
Recall (train):  0.8258706467661692


Tập test

In [18]:
y_test_pred = np.zeros(X_test.shape[0])
# print(X_train.iloc[0].values.reshape(-1, 1))
for i in range(X_test.shape[0]):
    x = X_test.iloc[i].values
    y_test_pred[i] = predict_NB_gaussian_class(x, mu_list, std_list, pi_list)
print('Accuracy (test): ', accuracy_score(y_test, y_test_pred))
print('Precision (test): ', precision_score(y_test, y_test_pred, pos_label=4))
print('Recall (test): ', recall_score(y_test, y_test_pred, pos_label=4))

Accuracy (test):  0.9666666666666667
Precision (test):  1.0
Recall (test):  0.9


## Sử dụng thư viện Scikit Learn


In [20]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)


sklearn_y_test_pred = model.predict(X_test)

print(accuracy_score(sklearn_y_test_pred, y_test))
print(precision_score(sklearn_y_test_pred, y_test, pos_label=4))
print(recall_score(sklearn_y_test_pred, y_test, pos_label=4))

0.9916666666666667
1.0
0.975609756097561


# Câu 3: Dự đoán loại ẩm thực

Đọc dữ liệu từ file csv

In [21]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week_1/data/multinomial_nb_orders.csv')
print(df.head(5))

      cuisine     city device user_segment    promo fav_ingredient  \
0    american  city_16    web          new  promo_1          onion   
1        thai  city_10    ios       family  promo_7       cilantro   
2    american  city_18    ios      student  promo_1         cheese   
3    japanese  city_10    web       family  promo_3         ginger   
4  vietnamese  city_10    web       family     none     fish_sauce   

  spice_level day_of_week price_bucket  
0         hot         Wed         high  
1        mild         Wed          mid  
2         hot         Fri          mid  
3        mild         Tue         high  
4        mild         Wed          low  


Tính xác suất P(c) = Nc/N

In [22]:
Y = df['cuisine']
y_labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype)
i = 0
for label in set(Y):
  y_labels[i, 0] = label
  y_labels[i, 1] = (Y == label).sum() / len(Y)
  i+=1


Tính toán xác suất P(xi|k) cùng với phương pháp Laplace Smoothing

In [24]:
def get_unique_value_in_col(df: pd.DataFrame):
  #Lấy ra tên của các trường trong DataFrame
  col_names = df.columns.values
  #Tạo một mảng 2 chiều lưu trữ tên cột và số giá trị phân biệt của cột đó
  unique_values = np.zeros((len(col_names), 2), dtype=col_names.dtype)
  #Đặt cột đầu tiên là tên của các cột
  unique_values[:, 0] = col_names

  #Lấy ra số lượng các giá trị phân biệt trong từng cột rồi gán vào cột thứ 2
  for i in range(len(col_names)):
    unique = set(df[col_names[i]])
    unique_values[i, 1] = len(unique)

  return unique_values

num_labels_M = get_unique_value_in_col(df=df)
print(num_labels_M)

def pxik_for_feature_each_class(X, xi, num_nomials_M, alpha = 1.0):
  X = np.array(X)
  count = (X==xi).sum()
  return (count + alpha)/(len(X) + num_nomials_M*alpha)

[['cuisine' 10]
 ['city' 20]
 ['device' 5]
 ['user_segment' 6]
 ['promo' 9]
 ['fav_ingredient' 18]
 ['spice_level' 5]
 ['day_of_week' 7]
 ['price_bucket' 4]]


Dự đoán output đầu ra

In [26]:
def predict_output_label(df, X, x_input, p_labels, target_feature):
  num_labels_M = get_unique_value_in_col(df)
  #Mảng p chứa loga của xác suất tiên nghiệm
  p = np.log(np.array(p_labels[:, 1], dtype=float))
  for k in range(len(p)):
    #Xk[c] chứa toàn bộ dữ liệu thuộc lớp c
    Xk = X[X[target_feature] == p_labels[k, 0]]
    for i in range(len(x_input)):
      #Công thức xác suất dự đoán sử dụng Laplace Smoothing
      p[k] += np.log(pxik_for_feature_each_class(Xk.iloc[:, i+1], x_input.iloc[i], num_labels_M[i + 1, 1]))
  
  y_star = np.argmax(p)
  return p_labels[y_star, 0]

Chia train test

In [37]:
train_size = round(len(df) * 0.7)
target_col = 'cuisine'
X_train = df[:train_size]
y_train = df[target_col][:train_size]

X_test = df[train_size:].drop(target_col,axis=1).reset_index(drop=True, inplace=False)
y_test = df[target_col][train_size:].reset_index(drop=True, inplace=False)

y_pred = np.zeros(len(y_test), dtype=y_test.dtype)

for i in range(len(X_test)):
  y_pred[i] = predict_output_label(df, X_train, X_test.iloc[i, :], y_labels, target_feature=target_col)



In [28]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_test))
print(predict_output_label(df, X_train, X_train.iloc[3, 1:], y_labels, target_col))

0.49
chinese


# Sử dụng thư viện Scikit Learn

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [49]:
X_train_lib = X_train.drop(target_col, axis=1, inplace=False)
# print(X_train_lib)
encoder1 = OneHotEncoder()
X_train_encoded = encoder1.fit_transform(X_train_lib)
X_test_encoded = encoder1.transform(X_test)

encoder2 = LabelEncoder()
y_train_encoded = encoder2.fit_transform(y_train)
y_test_encoded = encoder2.transform(y_test)

model = MultinomialNB()
model.fit(X_train_encoded, y_train_encoded)
y_pred = model.predict(X_test_encoded)
print('accuracy_score:', accuracy_score(y_test_encoded, y_pred))

accuracy_score: 0.4866666666666667
